<a href="https://colab.research.google.com/github/Mayank0195/IMDB_Web_Scraping/blob/main/mystery_movie_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
from time import sleep
from random import randint

In [2]:
#Declaring the headers
header = {"Accept-Language" : "en-US, en; q = 0.5"}

In [3]:
movie_name = []
year = []
time=[]
rating=[]
genres =[]
description = []
director = []
casts = []
images = []
number = []

In [4]:
#creating an array of values and passing it in the url for dynamic webpages
pages = list(np.arange(1,10000,50))

#the whole core of the script
for page in pages:
    pag = requests.get("https://www.imdb.com/search/title/?title_type=feature&genres=mystery&start={}&explore=genres&ref_=adv_nxt".format(page))
    soup = BeautifulSoup(pag.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
            name = store.find('h3', class_= 'lister-item-header').find('a').text
            movie_name.append(name)

            release = store.find('h3', class_= 'lister-item-header')
            try:
              year_of_release = store.find('span', class_ = "lister-item-year text-muted unbold").text[1:5]
              year.append(year_of_release)
            except:
              year.append(None)

            try: 
              runtime = store.p.find('span', class_ = 'runtime').text[0:7]
              time.append(runtime)
            except:
              time.append(None)

            rate = store.find('div', class_ = "inline-block ratings-imdb-rating")
            try:
              ratings = (rate.find('strong').text)
              rating.append(ratings)
            except:
              rating.append(None)

            genre = store.p.find('span', class_ = 'genre').text.replace('\n', '')
            genres.append(genre)

            describe = store.find_all('p', class_ = 'text-muted')
            description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
            description.append(description_)
            
            try:
              direct = store.select_one("p:nth-of-type(3)").find('a').text[:]
              director.append(direct)
            except:
              director.append(None)

            try:
              cast = store.select_one("p:nth-of-type(3)").select_one("a:nth-of-type(2)").text[:],store.select_one("p:nth-of-type(3)").select_one("a:nth-of-type(3)").text[:]
              casts.append(cast)
            except:
              casts.append(None)

            try:
              image = store.find('img')
              imge = image['loadlate']
              images.append(imge)  
            except:
                images.append(None)

            numbers = store.find('img')
            num = numbers['data-tconst'][:]
            try:
              number.append(num)
            except:
             number.append(None)

In [5]:
a = { "Movie Name": movie_name, "IMDB ID":number, "Year of Release" : year, "Watch Time": time,"Movie Rating": rating, "Genres": genres, "Description":description, "Director":director, "Cast":casts, "IMDB Image":images}

In [6]:
df = pd.DataFrame.from_dict(a, orient='index')
df = df.transpose()
df.head()

,Movie Name,IMDB ID,Year of Release,Watch Time,Movie Rating,Genres,Description,Director,Cast,IMDB Image
0,Don't Worry Darling,tt10731256,I) (,123 min,6.3,"Drama, Mystery, Thriller",A 1950s housewife living with her husband in a...,Olivia Wilde,"(Florence Pugh, Harry Styles)",https://m.media-amazon.com/images/M/MV5BMzFkMW...
1,Hellraiser,tt0887261,2022,None,None,"Horror, Mystery, Thriller",A take on Clive Barker's 1987 horror classic w...,David Bruckner,"(Jamie Clayton, Odessa A’zion)",https://m.media-amazon.com/images/M/MV5BYWM1YT...
2,X,tt13560574,II),105 min,6.6,"Horror, Mystery, Thriller","In 1979, a group of young filmmakers set out t...",Ti West,"(Mia Goth, Jenna Ortega)",https://m.media-amazon.com/images/M/MV5BMTJiMm...
3,Knock at the Cabin,tt15679400,2023,None,None,"Horror, Mystery, Thriller","While vacationing, a girl and her parents are ...",M. Night Shyamalan,"(Dave Bautista, Jonathan Groff)",https://m.media-amazon.com/images/M/MV5BNDIzMG...
4,Nope,tt10954984,2022,130 min,7.0,"Horror, Mystery, Sci-Fi",The residents of a lonely gulch in inland Cali...,Jordan Peele,"(Daniel Kaluuya, Keke Palmer)",https://m.media-amazon.com/images/M/MV5BNGM1MD...


In [7]:
df.shape

(9999, 10)

In [8]:
df['Movie Name'].value_counts()

The Hound of the Baskervilles    7
Blackout                         5
Seven Keys to Baldpate           4
Hide and Seek                    4
Spiral                           4
                                ..
Philosophy in the Boudoir        1
The Sublet                       1
Nails                            1
Connected                        1
A Dead Dame in Hollywood         1
Name: Movie Name, Length: 9608, dtype: int64

In [9]:
df = df.dropna()
df

,Movie Name,IMDB ID,Year of Release,Watch Time,Movie Rating,Genres,Description,Director,Cast,IMDB Image
0,Don't Worry Darling,tt10731256,I) (,123 min,6.3,"Drama, Mystery, Thriller",A 1950s housewife living with her husband in a...,Olivia Wilde,"(Florence Pugh, Harry Styles)",https://m.media-amazon.com/images/M/MV5BMzFkMW...
2,X,tt13560574,II),105 min,6.6,"Horror, Mystery, Thriller","In 1979, a group of young filmmakers set out t...",Ti West,"(Mia Goth, Jenna Ortega)",https://m.media-amazon.com/images/M/MV5BMTJiMm...
4,Nope,tt10954984,2022,130 min,7.0,"Horror, Mystery, Sci-Fi",The residents of a lonely gulch in inland Cali...,Jordan Peele,"(Daniel Kaluuya, Keke Palmer)",https://m.media-amazon.com/images/M/MV5BNGM1MD...
5,Where the Crawdads Sing,tt9411972,2022,125 min,7.1,"Drama, Mystery, Thriller",A woman who raised herself in the marshes of t...,Olivia Newman,"(Daisy Edgar-Jones, Taylor John Smith)",https://m.media-amazon.com/images/M/MV5BMTJmNG...
6,See How They Run,tt13640696,2022,98 min,6.7,"Comedy, Mystery","In the West End of 1950s London, plans for a m...",Tom George,"(Kieran Hodgson, Pearl Chanda)",https://m.media-amazon.com/images/M/MV5BNzU5OT...
...,...,...,...,...,...,...,...,...,...,...
9990,Köpekler Adasi,tt0286768,1997,73 min,6.7,"Drama, Mystery, Romance","The dramatic story of Esref, a journalist who ...",Halit Refig,"(Turan Alak, Mürsit Bag)",https://m.media-amazon.com/images/M/MV5BODhkYm...
9992,Crying Wolf,tt3389736,2013,55 min,6.1,Mystery,A college student thinks her roommate has been...,Jonathan May,"(Kate Ford, David Burgio)",https://m.media-amazon.com/images/M/MV5BMTM4Nz...
9993,Flic,tt0459804,2005,154 min,6.8,"Crime, Drama, Mystery",Kazuo Murata is a detective who has lost his w...,Masahiro Kobayashi,"(Teruyuki Kagawa, Seiichi Tanabe)",https://m.media-amazon.com/images/M/MV5BZWE3ZW...
9996,Weekend [Uik-end],tt4183002,2013,95 min,5.9,"Crime, Drama, Mystery",The main character of the movie - Igor Lebedev...,Stanislav Govorukhin,"(Maksim Matveev, Yuliya Khlynina)",https://m.media-amazon.com/images/M/MV5BOTE2Zj...


In [10]:
df.sort_values("Movie Name", inplace=True)
df

,Movie Name,IMDB ID,Year of Release,Watch Time,Movie Rating,Genres,Description,Director,Cast,IMDB Image
2407,#Horror,tt3526286,2015,97 min,3.0,"Crime, Drama, Horror",Six preadolescent girls face a night of terror...,Tara Subkoff,"(Sadie Seelert, Haley Murphy)",https://m.media-amazon.com/images/M/MV5BMjM5OD...
4013,#Screamers,tt5531032,2016,85 min,4.3,"Horror, Mystery",When an internet company decides to investigat...,Dean Matthew Ronalds,"(Tom Malloy, Chris Bannow)",https://m.media-amazon.com/images/M/MV5BNWYwMj...
7657,'Rameau's Nephew' by Diderot (Thanx to Dennis ...,tt0130430,1974,255 min,7.0,"Comedy, Mystery",Snow's ambitious attempt at exploring the form...,Michael Snow,"(Chantal Akerman, Steve Anker)",https://m.media-amazon.com/images/M/MV5BNWFhZG...
9077,.hack//G.U. Trilogy,tt1164545,2007,93 min,5.4,"Animation, Action, Adventure",After one of his friends falls into a coma pla...,Hiroshi Matsuyama,"(Takahiro Sakurai, Ayako Kawasumi)",https://m.media-amazon.com/images/M/MV5BODU4Nz...
5673,1 Versus 100,tt5322692,2019,78 min,7.3,"Mystery, Thriller","After Malia's father mysteriously dies, she mu...",Bruno Kohfield-Galeano,"(Anna McClean, Nancy Nezbit)",https://m.media-amazon.com/images/M/MV5BNjAwY2...
...,...,...,...,...,...,...,...,...,...,...
6757,"Ángela Morante, ¿crimen o suicidio?",tt0078546,1981,98 min,5.6,"Drama, Mystery",Add a Plot,José Estrada,"(Blanca Baldó, Enrique Lizalde)",https://m.media-amazon.com/images/M/MV5BNTc0Mj...
9053,Ángeles y querubines,tt0069549,1972,90 min,6.8,"Drama, Fantasy, Horror",A tragic celestial western that explores a sur...,Rafael Corkidi,"(Roberto Cañedo, Jorge Humberto Robles)",https://m.media-amazon.com/images/M/MV5BM2Y2Y2...
3401,Çocuklar Sana Emanet,tt7748432,2018,105 min,4.8,"Drama, Fantasy, Mystery","Kerem, who is a successful interior designer, ...",Cagan Irmak,"(Engin Akyürek, Hilal Altinbilek)",https://m.media-amazon.com/images/M/MV5BOTc5Nz...
8091,Écoute le temps,tt0443354,2006,87 min,5.8,"Crime, Drama, Mystery",The sound engineer Charlotte returns from Pari...,Alante Kavaite,"(Émilie Dequenne, Mathieu Demy)",https://m.media-amazon.com/images/M/MV5BMjA4MD...


In [11]:
df = df.drop_duplicates(subset=['IMDB ID'])
df

,Movie Name,IMDB ID,Year of Release,Watch Time,Movie Rating,Genres,Description,Director,Cast,IMDB Image
2407,#Horror,tt3526286,2015,97 min,3.0,"Crime, Drama, Horror",Six preadolescent girls face a night of terror...,Tara Subkoff,"(Sadie Seelert, Haley Murphy)",https://m.media-amazon.com/images/M/MV5BMjM5OD...
4013,#Screamers,tt5531032,2016,85 min,4.3,"Horror, Mystery",When an internet company decides to investigat...,Dean Matthew Ronalds,"(Tom Malloy, Chris Bannow)",https://m.media-amazon.com/images/M/MV5BNWYwMj...
7657,'Rameau's Nephew' by Diderot (Thanx to Dennis ...,tt0130430,1974,255 min,7.0,"Comedy, Mystery",Snow's ambitious attempt at exploring the form...,Michael Snow,"(Chantal Akerman, Steve Anker)",https://m.media-amazon.com/images/M/MV5BNWFhZG...
9077,.hack//G.U. Trilogy,tt1164545,2007,93 min,5.4,"Animation, Action, Adventure",After one of his friends falls into a coma pla...,Hiroshi Matsuyama,"(Takahiro Sakurai, Ayako Kawasumi)",https://m.media-amazon.com/images/M/MV5BODU4Nz...
5673,1 Versus 100,tt5322692,2019,78 min,7.3,"Mystery, Thriller","After Malia's father mysteriously dies, she mu...",Bruno Kohfield-Galeano,"(Anna McClean, Nancy Nezbit)",https://m.media-amazon.com/images/M/MV5BNjAwY2...
...,...,...,...,...,...,...,...,...,...,...
6757,"Ángela Morante, ¿crimen o suicidio?",tt0078546,1981,98 min,5.6,"Drama, Mystery",Add a Plot,José Estrada,"(Blanca Baldó, Enrique Lizalde)",https://m.media-amazon.com/images/M/MV5BNTc0Mj...
9053,Ángeles y querubines,tt0069549,1972,90 min,6.8,"Drama, Fantasy, Horror",A tragic celestial western that explores a sur...,Rafael Corkidi,"(Roberto Cañedo, Jorge Humberto Robles)",https://m.media-amazon.com/images/M/MV5BM2Y2Y2...
3401,Çocuklar Sana Emanet,tt7748432,2018,105 min,4.8,"Drama, Fantasy, Mystery","Kerem, who is a successful interior designer, ...",Cagan Irmak,"(Engin Akyürek, Hilal Altinbilek)",https://m.media-amazon.com/images/M/MV5BOTc5Nz...
8091,Écoute le temps,tt0443354,2006,87 min,5.8,"Crime, Drama, Mystery",The sound engineer Charlotte returns from Pari...,Alante Kavaite,"(Émilie Dequenne, Mathieu Demy)",https://m.media-amazon.com/images/M/MV5BMjA4MD...


In [12]:
df = df.reset_index(drop=True)
df

,Movie Name,IMDB ID,Year of Release,Watch Time,Movie Rating,Genres,Description,Director,Cast,IMDB Image
0,#Horror,tt3526286,2015,97 min,3.0,"Crime, Drama, Horror",Six preadolescent girls face a night of terror...,Tara Subkoff,"(Sadie Seelert, Haley Murphy)",https://m.media-amazon.com/images/M/MV5BMjM5OD...
1,#Screamers,tt5531032,2016,85 min,4.3,"Horror, Mystery",When an internet company decides to investigat...,Dean Matthew Ronalds,"(Tom Malloy, Chris Bannow)",https://m.media-amazon.com/images/M/MV5BNWYwMj...
2,'Rameau's Nephew' by Diderot (Thanx to Dennis ...,tt0130430,1974,255 min,7.0,"Comedy, Mystery",Snow's ambitious attempt at exploring the form...,Michael Snow,"(Chantal Akerman, Steve Anker)",https://m.media-amazon.com/images/M/MV5BNWFhZG...
3,.hack//G.U. Trilogy,tt1164545,2007,93 min,5.4,"Animation, Action, Adventure",After one of his friends falls into a coma pla...,Hiroshi Matsuyama,"(Takahiro Sakurai, Ayako Kawasumi)",https://m.media-amazon.com/images/M/MV5BODU4Nz...
4,1 Versus 100,tt5322692,2019,78 min,7.3,"Mystery, Thriller","After Malia's father mysteriously dies, she mu...",Bruno Kohfield-Galeano,"(Anna McClean, Nancy Nezbit)",https://m.media-amazon.com/images/M/MV5BNjAwY2...
...,...,...,...,...,...,...,...,...,...,...
8870,"Ángela Morante, ¿crimen o suicidio?",tt0078546,1981,98 min,5.6,"Drama, Mystery",Add a Plot,José Estrada,"(Blanca Baldó, Enrique Lizalde)",https://m.media-amazon.com/images/M/MV5BNTc0Mj...
8871,Ángeles y querubines,tt0069549,1972,90 min,6.8,"Drama, Fantasy, Horror",A tragic celestial western that explores a sur...,Rafael Corkidi,"(Roberto Cañedo, Jorge Humberto Robles)",https://m.media-amazon.com/images/M/MV5BM2Y2Y2...
8872,Çocuklar Sana Emanet,tt7748432,2018,105 min,4.8,"Drama, Fantasy, Mystery","Kerem, who is a successful interior designer, ...",Cagan Irmak,"(Engin Akyürek, Hilal Altinbilek)",https://m.media-amazon.com/images/M/MV5BOTc5Nz...
8873,Écoute le temps,tt0443354,2006,87 min,5.8,"Crime, Drama, Mystery",The sound engineer Charlotte returns from Pari...,Alante Kavaite,"(Émilie Dequenne, Mathieu Demy)",https://m.media-amazon.com/images/M/MV5BMjA4MD...


In [13]:
df[df['Movie Name'] == "Zack Snyder's Justice League"]

,Movie Name,IMDB ID,Year of Release,Watch Time,Movie Rating,Genres,Description,Director,Cast,IMDB Image


In [14]:
df['IMDB Image'][0]

'https://m.media-amazon.com/images/M/MV5BMjM5ODg2NjU2Nl5BMl5BanBnXkFtZTgwMTY0NTIwMDI@._V1_UY98_CR0,0,67,98_AL_.jpg'

In [15]:
df.to_csv('mystery.csv')

In [16]:
pag = requests.get("https://www.imdb.com/search/title/?title_type=feature&genres=action&after=WzEwNTUyMCwidHQwMTU1NzMzIiwxMDQ1MV0%3D&explore=genres&ref_=adv_nxt")
soup = BeautifulSoup(pag.text, 'html.parser')
movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
sleep(randint(2,8))
for store in movie_data:
  name = store.find('h3', class_= 'lister-item-header').find('a').text
  movie_name.append(name)

  release = store.find('h3', class_= 'lister-item-header')
  try:
    year_of_release = store.find('span', class_ = "lister-item-year text-muted unbold").text[1:5]
    year.append(year_of_release)
  except:
    year.append(None)

  try: 
     runtime = store.p.find('span', class_ = 'runtime').text[0:7]
     time.append(runtime)
  except:
      time.append(None)

  rate = store.find('div', class_ = "inline-block ratings-imdb-rating")
  try:
      ratings = (rate.find('strong').text)
      rating.append(ratings)
  except:
       rating.append(None)

  genre = store.p.find('span', class_ = 'genre').text.replace('\n', '')
  genres.append(genre)

  describe = store.find_all('p', class_ = 'text-muted')
  description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
  description.append(description_)
            
  try:
    direct = store.select_one("p:nth-of-type(3)").find('a').text[:]
    director.append(direct)
  except:
    director.append(None)

  try:
      cast = store.select_one("p:nth-of-type(3)").select_one("a:nth-of-type(2)").text[:],store.select_one("p:nth-of-type(3)").select_one("a:nth-of-type(3)").text[:]
      casts.append(cast)
  except:
      casts.append(None)

  try:
      image = store.find('img')
      imge = image['loadlate']
      images.append(imge)  
  except:
       images.append(None)

  numbers = store.find('img')
  num = numbers['data-tconst'][:]
  try:
    number.append(num)
  except:
    number.append(None)

In [17]:
b = { "Movie Name": movie_name, "IMDB ID":number, "Year of Release" : year, "Watch Time": time,"Movie Rating": rating, "Genres": genres, "Description":description, "Director":director, "Cast":casts, "IMDB Image":images}

In [18]:
df1 = pd.DataFrame.from_dict(b, orient='index')
df1 = df1.transpose()
df1.head()

,Movie Name,IMDB ID,Year of Release,Watch Time,Movie Rating,Genres,Description,Director,Cast,IMDB Image
0,Don't Worry Darling,tt10731256,I) (,123 min,6.3,"Drama, Mystery, Thriller",A 1950s housewife living with her husband in a...,Olivia Wilde,"(Florence Pugh, Harry Styles)",https://m.media-amazon.com/images/M/MV5BMzFkMW...
1,Hellraiser,tt0887261,2022,None,None,"Horror, Mystery, Thriller",A take on Clive Barker's 1987 horror classic w...,David Bruckner,"(Jamie Clayton, Odessa A’zion)",https://m.media-amazon.com/images/M/MV5BYWM1YT...
2,X,tt13560574,II),105 min,6.6,"Horror, Mystery, Thriller","In 1979, a group of young filmmakers set out t...",Ti West,"(Mia Goth, Jenna Ortega)",https://m.media-amazon.com/images/M/MV5BMTJiMm...
3,Knock at the Cabin,tt15679400,2023,None,None,"Horror, Mystery, Thriller","While vacationing, a girl and her parents are ...",M. Night Shyamalan,"(Dave Bautista, Jonathan Groff)",https://m.media-amazon.com/images/M/MV5BNDIzMG...
4,Nope,tt10954984,2022,130 min,7.0,"Horror, Mystery, Sci-Fi",The residents of a lonely gulch in inland Cali...,Jordan Peele,"(Daniel Kaluuya, Keke Palmer)",https://m.media-amazon.com/images/M/MV5BNGM1MD...


In [19]:
df1.shape

(10049, 10)

In [20]:
df1 = df1.dropna()
df1

,Movie Name,IMDB ID,Year of Release,Watch Time,Movie Rating,Genres,Description,Director,Cast,IMDB Image
0,Don't Worry Darling,tt10731256,I) (,123 min,6.3,"Drama, Mystery, Thriller",A 1950s housewife living with her husband in a...,Olivia Wilde,"(Florence Pugh, Harry Styles)",https://m.media-amazon.com/images/M/MV5BMzFkMW...
2,X,tt13560574,II),105 min,6.6,"Horror, Mystery, Thriller","In 1979, a group of young filmmakers set out t...",Ti West,"(Mia Goth, Jenna Ortega)",https://m.media-amazon.com/images/M/MV5BMTJiMm...
4,Nope,tt10954984,2022,130 min,7.0,"Horror, Mystery, Sci-Fi",The residents of a lonely gulch in inland Cali...,Jordan Peele,"(Daniel Kaluuya, Keke Palmer)",https://m.media-amazon.com/images/M/MV5BNGM1MD...
5,Where the Crawdads Sing,tt9411972,2022,125 min,7.1,"Drama, Mystery, Thriller",A woman who raised herself in the marshes of t...,Olivia Newman,"(Daisy Edgar-Jones, Taylor John Smith)",https://m.media-amazon.com/images/M/MV5BMTJmNG...
6,See How They Run,tt13640696,2022,98 min,6.7,"Comedy, Mystery","In the West End of 1950s London, plans for a m...",Tom George,"(Kieran Hodgson, Pearl Chanda)",https://m.media-amazon.com/images/M/MV5BNzU5OT...
...,...,...,...,...,...,...,...,...,...,...
10039,Son of Sardaar,tt2178508,2012,141 min,4.1,"Action, Comedy, Romance",A guest tries to stay inside the house of a ma...,Ashwani Dhir,"(Anil Devgan, Ajay Devgn)",https://m.media-amazon.com/images/M/MV5BMTkzMD...
10040,Das Frauenhaus,tt0167165,1977,75 min,4.6,"Action, Crime, Thriller",Nightclub owner and erotic dancer Rita Blue us...,Jesús Franco,"(Martine Fléty, Sarah Strasberg)",https://m.media-amazon.com/images/M/MV5BMTQwNj...
10041,Watch Movie Free Online,tt10096422,2019,45 min,7.1,"Action, Adventure, Comedy",The local TV director present new low budget p...,Sergey A.,"(Sergey A., Mykola Yeriomin)",https://m.media-amazon.com/images/M/MV5BODJmZD...
10044,Koyelaanchal,tt3640942,2014,145 min,5.4,"Action, Drama",Koyelaanchal (coal belt of India) brings to li...,Ashu Trikha,"(Kannan Arunachalam, Biswanath Basu)",https://m.media-amazon.com/images/M/MV5BMTA2OD...


In [21]:
df1.sort_values("Movie Name", inplace=True)
df1

,Movie Name,IMDB ID,Year of Release,Watch Time,Movie Rating,Genres,Description,Director,Cast,IMDB Image
2407,#Horror,tt3526286,2015,97 min,3.0,"Crime, Drama, Horror",Six preadolescent girls face a night of terror...,Tara Subkoff,"(Sadie Seelert, Haley Murphy)",https://m.media-amazon.com/images/M/MV5BMjM5OD...
4013,#Screamers,tt5531032,2016,85 min,4.3,"Horror, Mystery",When an internet company decides to investigat...,Dean Matthew Ronalds,"(Tom Malloy, Chris Bannow)",https://m.media-amazon.com/images/M/MV5BNWYwMj...
7657,'Rameau's Nephew' by Diderot (Thanx to Dennis ...,tt0130430,1974,255 min,7.0,"Comedy, Mystery",Snow's ambitious attempt at exploring the form...,Michael Snow,"(Chantal Akerman, Steve Anker)",https://m.media-amazon.com/images/M/MV5BNWFhZG...
9077,.hack//G.U. Trilogy,tt1164545,2007,93 min,5.4,"Animation, Action, Adventure",After one of his friends falls into a coma pla...,Hiroshi Matsuyama,"(Takahiro Sakurai, Ayako Kawasumi)",https://m.media-amazon.com/images/M/MV5BODU4Nz...
5673,1 Versus 100,tt5322692,2019,78 min,7.3,"Mystery, Thriller","After Malia's father mysteriously dies, she mu...",Bruno Kohfield-Galeano,"(Anna McClean, Nancy Nezbit)",https://m.media-amazon.com/images/M/MV5BNjAwY2...
...,...,...,...,...,...,...,...,...,...,...
6757,"Ángela Morante, ¿crimen o suicidio?",tt0078546,1981,98 min,5.6,"Drama, Mystery",Add a Plot,José Estrada,"(Blanca Baldó, Enrique Lizalde)",https://m.media-amazon.com/images/M/MV5BNTc0Mj...
9053,Ángeles y querubines,tt0069549,1972,90 min,6.8,"Drama, Fantasy, Horror",A tragic celestial western that explores a sur...,Rafael Corkidi,"(Roberto Cañedo, Jorge Humberto Robles)",https://m.media-amazon.com/images/M/MV5BM2Y2Y2...
3401,Çocuklar Sana Emanet,tt7748432,2018,105 min,4.8,"Drama, Fantasy, Mystery","Kerem, who is a successful interior designer, ...",Cagan Irmak,"(Engin Akyürek, Hilal Altinbilek)",https://m.media-amazon.com/images/M/MV5BOTc5Nz...
8091,Écoute le temps,tt0443354,2006,87 min,5.8,"Crime, Drama, Mystery",The sound engineer Charlotte returns from Pari...,Alante Kavaite,"(Émilie Dequenne, Mathieu Demy)",https://m.media-amazon.com/images/M/MV5BMjA4MD...


In [22]:
df1 = df1.drop_duplicates()
df1

,Movie Name,IMDB ID,Year of Release,Watch Time,Movie Rating,Genres,Description,Director,Cast,IMDB Image
2407,#Horror,tt3526286,2015,97 min,3.0,"Crime, Drama, Horror",Six preadolescent girls face a night of terror...,Tara Subkoff,"(Sadie Seelert, Haley Murphy)",https://m.media-amazon.com/images/M/MV5BMjM5OD...
4013,#Screamers,tt5531032,2016,85 min,4.3,"Horror, Mystery",When an internet company decides to investigat...,Dean Matthew Ronalds,"(Tom Malloy, Chris Bannow)",https://m.media-amazon.com/images/M/MV5BNWYwMj...
7657,'Rameau's Nephew' by Diderot (Thanx to Dennis ...,tt0130430,1974,255 min,7.0,"Comedy, Mystery",Snow's ambitious attempt at exploring the form...,Michael Snow,"(Chantal Akerman, Steve Anker)",https://m.media-amazon.com/images/M/MV5BNWFhZG...
9077,.hack//G.U. Trilogy,tt1164545,2007,93 min,5.4,"Animation, Action, Adventure",After one of his friends falls into a coma pla...,Hiroshi Matsuyama,"(Takahiro Sakurai, Ayako Kawasumi)",https://m.media-amazon.com/images/M/MV5BODU4Nz...
5673,1 Versus 100,tt5322692,2019,78 min,7.3,"Mystery, Thriller","After Malia's father mysteriously dies, she mu...",Bruno Kohfield-Galeano,"(Anna McClean, Nancy Nezbit)",https://m.media-amazon.com/images/M/MV5BNjAwY2...
...,...,...,...,...,...,...,...,...,...,...
6757,"Ángela Morante, ¿crimen o suicidio?",tt0078546,1981,98 min,5.6,"Drama, Mystery",Add a Plot,José Estrada,"(Blanca Baldó, Enrique Lizalde)",https://m.media-amazon.com/images/M/MV5BNTc0Mj...
9053,Ángeles y querubines,tt0069549,1972,90 min,6.8,"Drama, Fantasy, Horror",A tragic celestial western that explores a sur...,Rafael Corkidi,"(Roberto Cañedo, Jorge Humberto Robles)",https://m.media-amazon.com/images/M/MV5BM2Y2Y2...
3401,Çocuklar Sana Emanet,tt7748432,2018,105 min,4.8,"Drama, Fantasy, Mystery","Kerem, who is a successful interior designer, ...",Cagan Irmak,"(Engin Akyürek, Hilal Altinbilek)",https://m.media-amazon.com/images/M/MV5BOTc5Nz...
8091,Écoute le temps,tt0443354,2006,87 min,5.8,"Crime, Drama, Mystery",The sound engineer Charlotte returns from Pari...,Alante Kavaite,"(Émilie Dequenne, Mathieu Demy)",https://m.media-amazon.com/images/M/MV5BMjA4MD...


In [23]:
df1 = df1.reset_index(drop=True)
df1

,Movie Name,IMDB ID,Year of Release,Watch Time,Movie Rating,Genres,Description,Director,Cast,IMDB Image
0,#Horror,tt3526286,2015,97 min,3.0,"Crime, Drama, Horror",Six preadolescent girls face a night of terror...,Tara Subkoff,"(Sadie Seelert, Haley Murphy)",https://m.media-amazon.com/images/M/MV5BMjM5OD...
1,#Screamers,tt5531032,2016,85 min,4.3,"Horror, Mystery",When an internet company decides to investigat...,Dean Matthew Ronalds,"(Tom Malloy, Chris Bannow)",https://m.media-amazon.com/images/M/MV5BNWYwMj...
2,'Rameau's Nephew' by Diderot (Thanx to Dennis ...,tt0130430,1974,255 min,7.0,"Comedy, Mystery",Snow's ambitious attempt at exploring the form...,Michael Snow,"(Chantal Akerman, Steve Anker)",https://m.media-amazon.com/images/M/MV5BNWFhZG...
3,.hack//G.U. Trilogy,tt1164545,2007,93 min,5.4,"Animation, Action, Adventure",After one of his friends falls into a coma pla...,Hiroshi Matsuyama,"(Takahiro Sakurai, Ayako Kawasumi)",https://m.media-amazon.com/images/M/MV5BODU4Nz...
4,1 Versus 100,tt5322692,2019,78 min,7.3,"Mystery, Thriller","After Malia's father mysteriously dies, she mu...",Bruno Kohfield-Galeano,"(Anna McClean, Nancy Nezbit)",https://m.media-amazon.com/images/M/MV5BNjAwY2...
...,...,...,...,...,...,...,...,...,...,...
8911,"Ángela Morante, ¿crimen o suicidio?",tt0078546,1981,98 min,5.6,"Drama, Mystery",Add a Plot,José Estrada,"(Blanca Baldó, Enrique Lizalde)",https://m.media-amazon.com/images/M/MV5BNTc0Mj...
8912,Ángeles y querubines,tt0069549,1972,90 min,6.8,"Drama, Fantasy, Horror",A tragic celestial western that explores a sur...,Rafael Corkidi,"(Roberto Cañedo, Jorge Humberto Robles)",https://m.media-amazon.com/images/M/MV5BM2Y2Y2...
8913,Çocuklar Sana Emanet,tt7748432,2018,105 min,4.8,"Drama, Fantasy, Mystery","Kerem, who is a successful interior designer, ...",Cagan Irmak,"(Engin Akyürek, Hilal Altinbilek)",https://m.media-amazon.com/images/M/MV5BOTc5Nz...
8914,Écoute le temps,tt0443354,2006,87 min,5.8,"Crime, Drama, Mystery",The sound engineer Charlotte returns from Pari...,Alante Kavaite,"(Émilie Dequenne, Mathieu Demy)",https://m.media-amazon.com/images/M/MV5BMjA4MD...


In [24]:
df = df.append(df1, ignore_index=False )

In [25]:
df

,Movie Name,IMDB ID,Year of Release,Watch Time,Movie Rating,Genres,Description,Director,Cast,IMDB Image
0,#Horror,tt3526286,2015,97 min,3.0,"Crime, Drama, Horror",Six preadolescent girls face a night of terror...,Tara Subkoff,"(Sadie Seelert, Haley Murphy)",https://m.media-amazon.com/images/M/MV5BMjM5OD...
1,#Screamers,tt5531032,2016,85 min,4.3,"Horror, Mystery",When an internet company decides to investigat...,Dean Matthew Ronalds,"(Tom Malloy, Chris Bannow)",https://m.media-amazon.com/images/M/MV5BNWYwMj...
2,'Rameau's Nephew' by Diderot (Thanx to Dennis ...,tt0130430,1974,255 min,7.0,"Comedy, Mystery",Snow's ambitious attempt at exploring the form...,Michael Snow,"(Chantal Akerman, Steve Anker)",https://m.media-amazon.com/images/M/MV5BNWFhZG...
3,.hack//G.U. Trilogy,tt1164545,2007,93 min,5.4,"Animation, Action, Adventure",After one of his friends falls into a coma pla...,Hiroshi Matsuyama,"(Takahiro Sakurai, Ayako Kawasumi)",https://m.media-amazon.com/images/M/MV5BODU4Nz...
4,1 Versus 100,tt5322692,2019,78 min,7.3,"Mystery, Thriller","After Malia's father mysteriously dies, she mu...",Bruno Kohfield-Galeano,"(Anna McClean, Nancy Nezbit)",https://m.media-amazon.com/images/M/MV5BNjAwY2...
...,...,...,...,...,...,...,...,...,...,...
8911,"Ángela Morante, ¿crimen o suicidio?",tt0078546,1981,98 min,5.6,"Drama, Mystery",Add a Plot,José Estrada,"(Blanca Baldó, Enrique Lizalde)",https://m.media-amazon.com/images/M/MV5BNTc0Mj...
8912,Ángeles y querubines,tt0069549,1972,90 min,6.8,"Drama, Fantasy, Horror",A tragic celestial western that explores a sur...,Rafael Corkidi,"(Roberto Cañedo, Jorge Humberto Robles)",https://m.media-amazon.com/images/M/MV5BM2Y2Y2...
8913,Çocuklar Sana Emanet,tt7748432,2018,105 min,4.8,"Drama, Fantasy, Mystery","Kerem, who is a successful interior designer, ...",Cagan Irmak,"(Engin Akyürek, Hilal Altinbilek)",https://m.media-amazon.com/images/M/MV5BOTc5Nz...
8914,Écoute le temps,tt0443354,2006,87 min,5.8,"Crime, Drama, Mystery",The sound engineer Charlotte returns from Pari...,Alante Kavaite,"(Émilie Dequenne, Mathieu Demy)",https://m.media-amazon.com/images/M/MV5BMjA4MD...


In [26]:
df1 = df1.drop_duplicates(subset=['IMDB ID'])
df1

,Movie Name,IMDB ID,Year of Release,Watch Time,Movie Rating,Genres,Description,Director,Cast,IMDB Image
0,#Horror,tt3526286,2015,97 min,3.0,"Crime, Drama, Horror",Six preadolescent girls face a night of terror...,Tara Subkoff,"(Sadie Seelert, Haley Murphy)",https://m.media-amazon.com/images/M/MV5BMjM5OD...
1,#Screamers,tt5531032,2016,85 min,4.3,"Horror, Mystery",When an internet company decides to investigat...,Dean Matthew Ronalds,"(Tom Malloy, Chris Bannow)",https://m.media-amazon.com/images/M/MV5BNWYwMj...
2,'Rameau's Nephew' by Diderot (Thanx to Dennis ...,tt0130430,1974,255 min,7.0,"Comedy, Mystery",Snow's ambitious attempt at exploring the form...,Michael Snow,"(Chantal Akerman, Steve Anker)",https://m.media-amazon.com/images/M/MV5BNWFhZG...
3,.hack//G.U. Trilogy,tt1164545,2007,93 min,5.4,"Animation, Action, Adventure",After one of his friends falls into a coma pla...,Hiroshi Matsuyama,"(Takahiro Sakurai, Ayako Kawasumi)",https://m.media-amazon.com/images/M/MV5BODU4Nz...
4,1 Versus 100,tt5322692,2019,78 min,7.3,"Mystery, Thriller","After Malia's father mysteriously dies, she mu...",Bruno Kohfield-Galeano,"(Anna McClean, Nancy Nezbit)",https://m.media-amazon.com/images/M/MV5BNjAwY2...
...,...,...,...,...,...,...,...,...,...,...
8911,"Ángela Morante, ¿crimen o suicidio?",tt0078546,1981,98 min,5.6,"Drama, Mystery",Add a Plot,José Estrada,"(Blanca Baldó, Enrique Lizalde)",https://m.media-amazon.com/images/M/MV5BNTc0Mj...
8912,Ángeles y querubines,tt0069549,1972,90 min,6.8,"Drama, Fantasy, Horror",A tragic celestial western that explores a sur...,Rafael Corkidi,"(Roberto Cañedo, Jorge Humberto Robles)",https://m.media-amazon.com/images/M/MV5BM2Y2Y2...
8913,Çocuklar Sana Emanet,tt7748432,2018,105 min,4.8,"Drama, Fantasy, Mystery","Kerem, who is a successful interior designer, ...",Cagan Irmak,"(Engin Akyürek, Hilal Altinbilek)",https://m.media-amazon.com/images/M/MV5BOTc5Nz...
8914,Écoute le temps,tt0443354,2006,87 min,5.8,"Crime, Drama, Mystery",The sound engineer Charlotte returns from Pari...,Alante Kavaite,"(Émilie Dequenne, Mathieu Demy)",https://m.media-amazon.com/images/M/MV5BMjA4MD...


In [29]:
df.to_csv('mystery.csv')

In [28]:
df = pd.read_csv('action_movies.csv')
df

FileNotFoundError: ignored

In [ ]:
pag = requests.get("https://www.imdb.com/search/title/?title_type=feature&genres=action&after=WzEwODQwOSwidHQwMzA1ODMxIiwxMDY1MV0%3D&explore=genres&ref_=adv_nxt")
soup = BeautifulSoup(pag.text, 'html.parser')
movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
sleep(randint(2,8))
for store in movie_data:
  name = store.find('h3', class_= 'lister-item-header').find('a').text
  movie_name.append(name)

  release = store.find('h3', class_= 'lister-item-header')
  try:
    year_of_release = store.find('span', class_ = "lister-item-year text-muted unbold").text[1:5]
    year.append(year_of_release)
  except:
    year.append(None)

  try: 
     runtime = store.p.find('span', class_ = 'runtime').text[0:7]
     time.append(runtime)
  except:
      time.append(None)

  rate = store.find('div', class_ = "inline-block ratings-imdb-rating")
  try:
      ratings = (rate.find('strong').text)
      rating.append(ratings)
  except:
       rating.append(None)

  genre = store.p.find('span', class_ = 'genre').text.replace('\n', '')
  genres.append(genre)

  describe = store.find_all('p', class_ = 'text-muted')
  description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
  description.append(description_)
            
  try:
    direct = store.select_one("p:nth-of-type(3)").find('a').text[:]
    director.append(direct)
  except:
    director.append(None)

  try:
      cast = store.select_one("p:nth-of-type(3)").select_one("a:nth-of-type(2)").text[:],store.select_one("p:nth-of-type(3)").select_one("a:nth-of-type(3)").text[:]
      casts.append(cast)
  except:
      casts.append(None)

  try:
      image = store.find('img')
      imge = image['loadlate']
      images.append(imge)  
  except:
       images.append(None)

  numbers = store.find('img')
  num = numbers['data-tconst'][:]
  try:
    number.append(num)
  except:
    number.append(None)

In [ ]:
c = { "Movie Name": movie_name, "IMDB ID":number, "Year of Release" : year, "Watch Time": time,"Movie Rating": rating, "Genres": genres, "Description":description, "Director":director, "Cast":casts, "IMDB Image":images}

In [ ]:
df2 = pd.DataFrame.from_dict(c, orient='index')
df2 = df2.transpose()
df2.head()

In [ ]:
df2.shape

In [ ]:
df2 = df2.dropna()
df2

In [ ]:
df2.sort_values("Movie Name", inplace=True)
df2

In [ ]:
df2 = df2.drop_duplicates(subset=['IMDB ID'])
df2

In [ ]:
df2 = df2.reset_index(drop=True)
df2

In [ ]:
df = df.append(df2, ignore_index=False  )
df

In [ ]:
df.shape

In [ ]:
df = df.drop_duplicates(subset=['IMDB ID'])
df

In [ ]:
df.to_csv('action_movies.csv')

In [ ]:
df[df['Movie Name'] == 'Tharangam']